In [1]:
import os
from dotenv import load_dotenv
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.schema import Document, SystemMessage, HumanMessage

In [2]:
load_dotenv()
api_key = "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"

In [3]:
chat = ChatGroq(temperature=0, groq_api_key=api_key, model_name="llama3-70b-8192")

In [4]:
embedding_model = HuggingFaceEmbeddings(model_name='sentence-transformers/paraphrase-MiniLM-L6-v2')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-07-22 18:15:15.618482: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical op

In [5]:
persist_directory = "../RAG_3_vectordb_3_separate codes/article_chroma_db"

ALWAYS CHECK IF THE DIRECTORY EXISTS

In [6]:
if not os.path.exists(persist_directory):
    print("Persist directory does not exist.")
else:
    print("Persist directory exists.")

Persist directory exists.


In [7]:
vectorstore = Chroma(persist_directory=persist_directory, embedding_function=embedding_model)

In [8]:
metadata_field_info = [
    AttributeInfo(
        name="article_id",
        description="Article ID of the paper",
        type="string",
    ),
    AttributeInfo(
        name="authors",
                description="Authors of the paper",
        type="string or list[string]",
    ),
    AttributeInfo(
        name="year",
        description="Year the paper was published",
        type="integer",
    ),
    AttributeInfo(
        name="abstract",
        description="Abstract of the article",
        type="string",
    ),
    AttributeInfo(
        name="title",
        description="Title of the paper",
        type="string",
    ),
    AttributeInfo(
        name="keywords",
        description="Keywords associated with the paper",
        type="string or list[string]",
    ),
    AttributeInfo(
        name="citation_count",
        description="Number of citations the paper has received",
        type="integer",
    )
]

document_content_description = "Provides information about article"

In [9]:
retriever = SelfQueryRetriever.from_llm(
    llm=chat,
    vectorstore=vectorstore,
    document_contents=document_content_description,
    metadata_field_info=metadata_field_info,
    verbose=True
)

In [10]:
custom_prompt_template = """Use the following pieces of information to answer the user's question. Always answear the question as if you were a human and answear in full sentance. During your answear be really specific. If you don't know the answer, just say that you don't know, don't try to make up an answer.



Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [11]:
def set_custom_prompt():
    """
    Prompt template for QA retrieval for each vectorstore
    """
    prompt = PromptTemplate(template=custom_prompt_template,
                            input_variables=['context', 'question'])
    return prompt

prompt = set_custom_prompt()

In [12]:
qa = RetrievalQA.from_chain_type(
    llm=chat,
    chain_type='stuff',
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={'prompt': prompt}
)

In [13]:
query = "How many articles were published in 2016"
result = qa({"query": query})
print("Answer:", result["result"])

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Answer: There were 2 articles published in 2016, both of which have a citation count of 0.


In [14]:
query = "Which article had citation count higher than 250"
result = qa({"query": query})
print("Answer:", result["result"])

Answer: All three articles have a citation count higher than 250. The first two articles have a citation count of 296, and the last two articles have a citation count of 299.


In [15]:
query = "Which sections does article A Multilevel Model of Resistance to Information Technology Implementation have"
result = qa({"query": query})
print("Answer:", result["result"])

Answer: The article "A Multilevel Model of Resistance to Information Technology Implementation" has the following sections: Abstract, Keywords, and Citation Count.


In [16]:
#yasmin:
query = "Which sections does article An Alternative to Methodological Individualism: A Non-Reductionist Approach to Studying Technology Adoption by Groups have"
result = qa({"query": query})
print("Answer:", result["result"])

Answer: The article "An Alternative to Methodological Individualism: A Non-Reductionist Approach to Studying Technology Adoption by Groups" has an abstract, keywords, and a citation count section.


In [17]:
#yasmin:
query = "Which section is the first in the article An Alternative to Methodological Individualism: A Non-Reductionist Approach to Studying Technology Adoption by Groups?"
result = qa({"query": query})
print("Answer:", result["result"])

Answer: The first section in the article "An Alternative to Methodological Individualism: A Non-Reductionist Approach to Studying Technology Adoption by Groups" is the Abstract.


In [18]:
#yasmin:
query = "Who is the author of the article An Alternative to Methodological Individualism: A Non-Reductionist Approach to Studying Technology Adoption by Groups?"
result = qa({"query": query})
print("Answer:", result["result"])

Answer: The authors of the article "An Alternative to Methodological Individualism: A Non-Reductionist Approach to Studying Technology Adoption by Groups" are Sarker, Saonee, and Valacich, Joseph S.


In [19]:
#yasmin:
query = "In which year was the article An Alternative to Methodological Individualism: A Non-Reductionist Approach to Studying Technology Adoption by Groups written?"
result = qa({"query": query})
print("Answer:", result["result"])

Answer: The article "An Alternative to Methodological Individualism: A Non-Reductionist Approach to Studying Technology Adoption by Groups" was written in 2010.


In [20]:
#yasmin:
query = "What is the Contribution to our understanding of IT use of the article An Alternative to Methodological Individualism: A Non-Reductionist Approach to Studying Technology Adoption by Groups written?"
result = qa({"query": query})
print("Answer:", result["result"])

Answer: The article "An Alternative to Methodological Individualism: A Non-Reductionist Approach to Studying Technology Adoption by Groups" contributes to our understanding of IT use by providing a non-reductionist approach to studying technology adoption by groups, which offers a more robust explanation of technology adoption than the traditional methodological individualist view, and highlights the conditions under which the methodological individualist view fails to provide correct explanations.


In [21]:
#yasmin:
query = "How was the use studied or conceptualized of the article An Alternative to Methodological Individualism: A Non-Reductionist Approach to Studying Technology Adoption by Groups written?"
result = qa({"query": query})
print("Answer:", result["result"])

Answer: The use of technology adoption by groups was conceptualized in the article "An Alternative to Methodological Individualism: A Non-Reductionist Approach to Studying Technology Adoption by Groups" through a non-reductionist model that drew on theories of group influence, prior research on conflict, technology characteristics, task-technology fit, group communication media, and recent theoretical work surrounding group technology adoption.


In [22]:
#yasmin:
query = "Briefly: What is the theory of the article An Alternative to Methodological Individualism: A Non-Reductionist Approach to Studying Technology Adoption by Groups written?"
result = qa({"query": query})
print("Answer:", result["result"])

Answer: The theory of the article "An Alternative to Methodological Individualism: A Non-Reductionist Approach to Studying Technology Adoption by Groups" is a non-reductionist approach to investigating group-related phenomenon, specifically in the context of technology adoption, which draws on theories of group influence, prior research on conflict, technology characteristics, task-technology fit, group communication media, and recent theoretical work surrounding group technology adoption.


In [23]:
#yasmin:
query = "In 5 words: What is the empirical approach of the article An Alternative to Methodological Individualism: A Non-Reductionist Approach to Studying Technology Adoption by Groups written?"
result = qa({"query": query})
print("Answer:", result["result"])

Answer: The empirical approach of the article "An Alternative to Methodological Individualism: A Non-Reductionist Approach to Studying Technology Adoption by Groups" is based on case studies.


In [24]:
#yasmin:
query = "Is there an article by the author Subramani?"
result = qa({"query": query})
print("Answer:", result["result"])

Answer: Subramani is a Fijian author, and yes, he has written several articles and books, but I don't have specific information on a particular article by him, could you please provide more context or details about the article you're looking for?


In [25]:
#yasmin:
query = "What is the Contribution to our understanding of IT use in the article Revisiting Group-Based Technology Adoption as a Dynamic Process: The Role of Changing Attitude-Rationale Configurations?"
result = qa({"query": query})
print("Answer:", result["result"])



Answer: The article "Revisiting Group-Based Technology Adoption as a Dynamic Process: The Role of Changing Attitude-Rationale Configurations" contributes to our understanding of IT use by highlighting the importance of collective decision-making and the role of subgroups in determining group technology adoption states, as well as the dynamic process of technology adoption rationales and attitudes, which can lead to the continuance or discontinuance of information systems use.


In [26]:
#yasmin:
query = "What is the theory in the article Revisiting Group-Based Technology Adoption as a Dynamic Process: The Role of Changing Attitude-Rationale Configurations?"
result = qa({"query": query})
print("Answer:", result["result"])

Answer: Unfortunately, the abstract provided does not explicitly state a specific theory in the article "Revisiting Group-Based Technology Adoption as a Dynamic Process: The Role of Changing Attitude-Rationale Configurations". It discusses various concepts and topics related to group-based technology adoption, but it does not mention a specific theory.


In [27]:
query = "For the article Revisiting Group-Based Technology Adoption as a Dynamic Process: The Role of Changing Attitude-Rationale Configurations, how was the use studied or conceptualized?"
result = qa({"query": query})
print("Answer:", result["result"])

Answer: The use of technology was studied or conceptualized through a qualitative approach for group technology adoption, specifically focusing on the roles of subgroups in determining group technology adoption states (TASs) and the continuance or discontinuance of information systems use.


In [31]:
query = "What is the theory behind the article Revisiting Group-Based Technology Adoption as a Dynamic Process: The Role of Changing Attitude-Rationale Configurations?"
result = qa({"query": query})
print("Answer:", result["result"])

Answer: Unfortunately, the provided abstract does not explicitly state the specific theory behind the article "Revisiting Group-Based Technology Adoption as a Dynamic Process: The Role of Changing Attitude-Rationale Configurations". However, based on the keywords and topics mentioned, it appears to be related to the Technology Adoption Model, which explores how users form attitudes and intentions towards adopting new technologies.


In [29]:
query = "What is the empirical approach of the article Revisiting Group-Based Technology Adoption as a Dynamic Process: The Role of Changing Attitude-Rationale Configurations?"
result = qa({"query": query})
print("Answer:", result["result"])

Answer: The empirical approach of the article "Revisiting Group-Based Technology Adoption as a Dynamic Process: The Role of Changing Attitude-Rationale Configurations" is a qualitative approach, specifically a case study.


In [33]:
query = "From which year is the article Revisiting Group-Based Technology Adoption as a Dynamic Process: The Role of Changing Attitude-Rationale Configurations?"
result = qa({"query": query})
print("Answer:", result["result"])

Answer: The article "Revisiting Group-Based Technology Adoption as a Dynamic Process: The Role of Changing Attitude-Rationale Configurations" is from the year 2016.


In [34]:
#yasmin:
query = "What is the Contribution to our understanding of IT use in the article Coping with Information Technology: Mixed Emotions, Vacillation, and Nonconforming Use Patterns?"
result = qa({"query": query})
print("Answer:", result["result"])

Answer: The article "Coping with Information Technology: Mixed Emotions, Vacillation, and Nonconforming Use Patterns" contributes to our understanding of IT use by highlighting the role of emotions in shaping IT use patterns, specifically how mixed emotions can lead to vacillating strategies and nonconforming use patterns that can have positive organizational implications, despite not conforming to the terms of use.


In [35]:
query = "For the article Coping with Information Technology: Mixed Emotions, Vacillation, and Nonconforming Use Patterns, how was the use studied or conceptualized?"
result = qa({"query": query})
print("Answer:", result["result"])

Answer: The use of IT was studied or conceptualized through an in-depth field study conducted in two North American universities, examining the role of emotions in how specific IT use patterns emerge.


In [36]:
query = "What is the theory behind the article Coping with Information Technology: Mixed Emotions, Vacillation, and Nonconforming Use Patterns?"
result = qa({"query": query})
print("Answer:", result["result"])

Answer: The theory behind the article "Coping with Information Technology: Mixed Emotions, Vacillation, and Nonconforming Use Patterns" is that there are five different characteristics of an IT stimulus event (cues) that, when interacting in a reinforcing manner, elicit a single class of emotions (uniform affective responses) and, when interacting in an oppositional manner, elicit mixed emotions (ambivalent affective responses), leading to different adaptation behaviors and use patterns.


In [38]:
query = "What is the empirical approach of the article Coping with Information Technology: Mixed Emotions, Vacillation, and Nonconforming Use Patterns?"
result = qa({"query": query})
print("Answer:", result["result"])

Answer: The empirical approach of the article "Coping with Information Technology: Mixed Emotions, Vacillation, and Nonconforming Use Patterns" is a qualitative research approach, as indicated by the keyword "qualitative research" in the abstract.
